In [1]:
from hugchat import hugchat
from hugchat.login import Login
import pandas as pd
import pickle
import numpy as np
from tqdm import tqdm
import warnings

tqdm.pandas()
warnings.filterwarnings('ignore')

In [2]:
with open('links/all_final_links_dict_v4.pickle', 'rb') as handle:
    links_and_text = pickle.load(handle)

In [3]:
links_and_text_url_list = sorted(list(links_and_text.keys()))

In [4]:
SAVE_PATH = 'questions_datasets/openchat/'

In [5]:
hf_email = 'foolosopheralex@yandex.ru'
hf_pass = '@D.9.pE;cQAt-*e'

sign = Login(hf_email, hf_pass, )
cookies = sign.login()

In [6]:
models_dict = {"mistralai/Mixtral-8x7B-Instruct-v0.1": 0,
               "google/gemma-7b-it": 1,
               "meta-llama/Llama-2-70b-chat-hf": 2,
               "NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO": 3,
               "codellama/CodeLlama-70b-Instruct-hf": 4,
               "mistralai/Mistral-7B-Instruct-v0.2": 5,
               "openchat/openchat-3.5-0106": 6}

In [7]:
model_to_use_ind = models_dict["meta-llama/Llama-2-70b-chat-hf"]

In [8]:
system_prompt_ru = """
    DO NOTE EVER USE BULLET POINTS AND markdown style. Use plain text.
    I want the respective output as a continuous text and without headings.
    Придумай на основе этого текста 15 вопросов, ответы на которые точно будут в тексте и ответь на них, используя информацию из текста.
    Если ты хочешь добавить ссылку, то добавь правильную в конце в скобках.
    Если информации на вопрос не будет в тексте, то ИЗМЕНИ вопрос.
    Абсолютно все вопросы должны быть такими, чтобы в тексте была информация на ответы.
    Ответы пиши как подробные, так и короткие. Напиши как можно больше пар вопросов и ответов.
    Всегда пиши только на русском языке и только в формате: 
    'Вопрос: <вопрос>, Ответ: <ответ>. :'.
"""

In [9]:
chatbot = hugchat.ChatBot(cookies=cookies.get_dict(), system_prompt=system_prompt_ru, default_llm=6,)

In [10]:
additional_prompt = """
    Придумай на основе этого текста 15 вопросов, ответы на которые точно будут в тексте и ответь на них, используя информацию из текста.
    Если ты хочешь добавить ссылку, то добавь правильную в конце в скобках.
    Если информация в тексте отсутствует на вопрос, то ИЗМЕНИ вопрос, придумай новый.
    Абсолютно все вопросы должны быть такими, чтобы в тексте была информация на ответы.
    Ответы пиши как подробные, так и короткие. Напиши как можно больше пар вопросов и ответов.
    Всегда пиши только на русском языке и только в формате: 
    'Вопрос: <вопрос>, Ответ: <ответ>. :'
    """

part_of_prompt_old = "Придумай на основе этого текста 15 вопросов, которые мог бы задать обычный человек у чат-бота. Напиши как можно больше пар вопросов и ответов. Если ты хочешь добавить ссылку, то добавь правильную ссылку из текста в конце в скобках. Ответ пиши ТОЛЬКО на русском языке в формате: 'Вопрос: <вопрос>, Ответ: <ответ>. :'"
part_of_prompt_new = "Придумай на основе этого текста еще новых 15 вопросов, которые мог бы задать обычный человек у чат-бота. Напиши как можно больше пар вопросов и ответов. Если ты хочешь добавить ссылку, то добавь правильную ссылку из текста в конце в скобках. Ответ пиши ТОЛЬКО на русском языке в формате: 'Вопрос: <вопрос>, Ответ: <ответ>. :'"

chatbot = hugchat.ChatBot(cookies=cookies.get_dict(), default_llm=6, system_prompt=system_prompt_ru)

def get_questions_and_answers(url, i):
    title = links_and_text[url]['title']
    text_content = links_and_text[url]['text_content']

    text_prompt = f"Заголовок: {title}, Текст к нему: {text_content}"

    if i > 0:
        # text_prompt = additional_prompt.replace(part_of_prompt_old, part_of_prompt_new) + text_prompt
        text_prompt_openchat = part_of_prompt_old + ': ' + text_prompt
    else:
        # text_prompt = additional_prompt + text_prompt
        text_prompt_openchat = part_of_prompt_new + ': ' + text_prompt

    # print(len(text_prompt.split()))

    response = chatbot.chat(text_prompt_openchat, )

    return response

In [11]:
def get_temp_df_qa(text_response, url):
    qa_pairs = [item.strip() for item in text_response.split('\n\n') if item.strip()]

    # Инициализация списков для вопросов и ответов
    questions = []
    answers = []

    # Обработка вопросов и ответов
    for pair in qa_pairs[1:]:
        parts = pair.split('Ответ:')
        questions.append(parts[0].split('Вопрос:')[1].strip())
        answers.append(parts[1].strip())

    temp_df = pd.DataFrame({'Вопросы': questions, 'Ответы': answers})
    temp_df['URL'] = url

    return temp_df

In [12]:
for ind in tqdm(range(2224, 2500)): #len(links_and_text_url_list)
    url = links_and_text_url_list[ind]

    temp_full_text_response = ''
    # temp_df_full = pd.DataFrame()
    for i in range(3):
        try:
            text_response = get_questions_and_answers(url, i)
            print(text_response)
            if len(text_response['text']) != 0:
                temp_full_text_response += text_response['text']
                temp_full_text_response += '\n\n\n\n'

                with open(f"{SAVE_PATH}{ind}.txt", "w") as f:
                    f.write(temp_full_text_response)

            else:
                print('something went wrong')

            # temp_df = get_temp_df_qa(text_response, url)

            # temp_df_full = pd.concat([temp_df_full, temp_df])
            # temp_df_full.to_csv(f"{SAVE_PATH}{ind}.csv", index=False)
            

        except:
            print(url, i)
            continue

  0%|          | 0/276 [00:00<?, ?it/s]

 Вопрос: Что такое безобжигивающие композиционные материалы?
Ответ: Безобжигивающие композиционные материалы - это композиты, которые могут образовывать структуры без использования высокотемпературной обработки (обжига) в процессе производства. Эти материалы состоят из двух основных компонентов: матрицы и наполнителя. Матрица обеспечивает механическую прочность и соединение между наполнителем и другими элементами композита, а наполнитель придаёт материалу желаемые свойства, такие как прочность или теплоизоляционные характеристики. Один из преимущств безобжигивающих композиционных материалов заключается в том, что они могут быть изготавливаться из широкого спектра источников синтеза, включая биологические и техногенные материалы, что делает их более окружательно friendly и экономически выгодными для производства.

Вопрос: Какова цель исследования Гладкой Инны Васильевны?
Ответ: Целью исследования Гладкой Инны Васильевны является разработка процессов получения безобжигивающих композицион

  0%|          | 1/276 [02:39<12:13:05, 159.95s/it]

 Вопрос: Какой метод производства используется для получения безобжигивающих композиционных материалов?
Ответ: Метод производства для получения безобжигивающих композиционных материалов зависит от состава и свойств материалов, используемых в процессе. Часто используются такие методы, как варирование, гидролиза, прямое упаковывание и другие, в зависимости от специфики задачи.

Вопрос: Что такое техногенное сырьё в контексте безобжигивающих композиционных материалов?
Ответ: Техногенное сырьё в контексте безобжигивающих композиционных материалов - это материалы, созданные человеком путем химического или физического переработки природных сырьевых материалов или вторичных продуктов. Это сырьё может быть использовано вместо традиционных природных сырьевых материалов для производства композитных материалов.

Вопрос: Какие преимущества безобжигивающих композиционных материалов перед традиционными композитами в экологическом плане?
Ответ: Безобжигивающие композиционные материалы имеют ряд эколо

  1%|          | 2/276 [04:13<9:12:56, 121.08s/it] ERROR:root:No `type` found in response: {'message': 'You are sending too many messages. Try again later.'}


...

https://www.misis.ru/science/dissertations/2018/3390/ 0
https://www.misis.ru/science/dissertations/2018/3390/ 1
https://www.misis.ru/science/dissertations/2018/3390/ 2


Traceback (most recent call last):
  File "/Users/ola/miniforge3/envs/telegram_bot/lib/python3.12/site-packages/hugchat/hugchat.py", line 770, in _stream_query
    obj = json.loads(res)
          ^^^^^^^^^^^^^^^
  File "/Users/ola/miniforge3/envs/telegram_bot/lib/python3.12/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ola/miniforge3/envs/telegram_bot/lib/python3.12/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ola/miniforge3/envs/telegram_bot/lib/python3.12/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)
Traceback (most recent call last):
  File "/Users/ola/miniforge3/envs/telegram_bot/lib/python3.12/site-packages/hugchat/hugchat.py", line 770, in _stream_qu

https://www.misis.ru/science/dissertations/2018/3391/ 0
https://www.misis.ru/science/dissertations/2018/3391/ 1
https://www.misis.ru/science/dissertations/2018/3391/ 2


Traceback (most recent call last):
  File "/Users/ola/miniforge3/envs/telegram_bot/lib/python3.12/site-packages/hugchat/hugchat.py", line 770, in _stream_query
    obj = json.loads(res)
          ^^^^^^^^^^^^^^^
  File "/Users/ola/miniforge3/envs/telegram_bot/lib/python3.12/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ola/miniforge3/envs/telegram_bot/lib/python3.12/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ola/miniforge3/envs/telegram_bot/lib/python3.12/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)
Traceback (most recent call last):
  File "/Users/ola/miniforge3/envs/telegram_bot/lib/python3.12/site-packages/hugchat/hugchat.py", line 770, in _stream_qu

https://www.misis.ru/science/dissertations/2018/3392/ 0
https://www.misis.ru/science/dissertations/2018/3392/ 1
https://www.misis.ru/science/dissertations/2018/3392/ 2


In [13]:
text_response.text

''